# Titanic

## Setup

In [12]:
# general imports
import numpy as np

# for project imports
import sys
from pathlib import Path

root = Path.cwd().parent
sys.path.append(str(root))

## Load Data

In [13]:
import pandas as pd
from src.data_utils import download_kaggle_competition

files = download_kaggle_competition("titanic")
print(files)

train_df = pd.read_csv(root / Path("data/raw/titanic/train.csv"))
test_df  = pd.read_csv(root / Path("data/raw/titanic/test.csv"))

X = train_df.drop(columns=["Survived", "Name", "Ticket", "Cabin"])
y = train_df["Survived"]

from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

[PosixPath('data/raw/titanic/gender_submission.csv'), PosixPath('data/raw/titanic/test.csv'), PosixPath('data/raw/titanic/train.csv')]


## Train Model

In [14]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV

numeric_features = ["Age", "SibSp", "Parch", "Fare"]
numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

categorical_features = ["Pclass", "Sex", "Embarked"]
categorical_transformer = Pipeline(steps=[
    ("onehot", OneHotEncoder(handle_unknown="ignore"))
])

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features)
    ]
)

clf = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("classifier", LogisticRegression(max_iter=1000))
])

param_grid = {
    "classifier__C": [0.01, 0.1, 1, 10, 100],
    "classifier__penalty": ["l2"],
    "classifier__solver": ["lbfgs"]
}

grid_search = GridSearchCV(clf, param_grid, cv=5, scoring="accuracy")
grid_search.fit(X_train, y_train)
print("Best parameters:", grid_search.best_params_)

best_model = grid_search.best_estimator_
y_val_pred = best_model.predict(X_val)
val_accuracy = accuracy_score(y_val, y_val_pred)
print("Validation Accuracy:", val_accuracy)


Best parameters: {'classifier__C': 0.1, 'classifier__penalty': 'l2', 'classifier__solver': 'lbfgs'}
Validation Accuracy: 0.7988826815642458


## Predict

In [15]:
# prdict on test set
X_test = test_df.drop(columns=["Name", "Ticket", "Cabin"])
y_test_pred = best_model.predict(X_test)
submission_df = pd.DataFrame({
    "PassengerId": test_df["PassengerId"],
    "Survived": y_test_pred
})
submission_df.to_csv(root / Path("data/processed/titanic_submission.csv"), index=False)